## PARTIE 1 : Scraper les informations nécessaires à partir du site Schlouk Map

In [1]:
import requests
from bs4 import BeautifulSoup

# URL de la page contenant les liens des bars
url_page = "https://www.schlouk-map.com/fr/cities/toulouse/happy-hour"

# Récupérer la page contenant les liens des bars
response_page = requests.get(url_page)
soup_page = BeautifulSoup(response_page.content, 'html.parser')

# Récupérer tous les liens des bars de la page
liens_bars = [a['href'] for a in soup_page.find_all('a', class_='name')]

# Afficher les liens des bars
for lien in liens_bars:
    print(lien)

/fr/places/matabiau-decapsule-club
/fr/places/tom-pouce
/fr/places/the-black-lion-1
/fr/places/barricot
/fr/places/la-maison-1
/fr/places/el-circo
/fr/places/o-boudu-pont
/fr/places/le-tchin
/fr/places/the-botanist-pub
/fr/places/le-pery
/fr/places/le-biergarten
/fr/places/cafe-oz-2
/fr/places/the-frog-rosbif-2
/fr/places/the-petit-london
/fr/places/the-danu
/fr/places/snapper-rock
/fr/places/the-hopscotch
/fr/places/le-bistrologue
/fr/places/1862-artybar
/fr/places/kraken-paradise
/fr/places/mad-meet-and-drink
/fr/places/le-delicatessen
/fr/places/bar-martini-club
/fr/places/borriquito-loco
/fr/places/lautruche
/fr/places/sauvage-social-pub
/fr/places/beer-mosaic
/fr/places/lalmanach-bistrot
/fr/places/le-saint-des-seins
/fr/places/la-tireuse
/fr/places/the-london-town
/fr/places/la-tantina-de-burgos
/fr/places/rooster-beer
/fr/places/cacahuete
/fr/places/au-fut-et-a-mesure-7
/fr/places/cafe-populaire


In [2]:
import requests
from bs4 import BeautifulSoup
import re

def extraire_informations_bar(href_bar):
    
    # Récupération de la page web du bar
    response = requests.get(href_bar)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Étape 1 : Récupérer le nom du bar
    nom_bar = soup.find('h1', class_='mb-0 d-inline-block').text.strip()

    # Étape 2 : Récupérer les heures d'ouverture
    heures_ouverture = soup.find('div', class_="col-lg-6 mb-3 mb-md-0").text.strip()

    # Étape 4 : Récupérer les services (uniquement oui)
    services = []
    table_services = soup.find('table', class_='table-services')
    if table_services:
        for row in table_services.find_all('tr'):
            service = row.find('td', class_='text-truncate').text.strip()
            disponibilite = row.find('td', class_='float-right').text.strip()
            if disponibilite == 'Oui':
                services.append(service)

    # Étape 5 : Récupérer les tarifs des boissons en happy hour/standard
    tarifs = soup.find('div', class_= "position-relative").text.strip()

    # Retourner les informations
    return {
        'Nom': nom_bar,
        'Heures_Ouverture': heures_ouverture,
        'Services': services,
        'Tarifs': tarifs
    }

# Liste pour stocker les informations extraites de chaque bar
informations_bars = []

# Itérer sur chaque lien de bar
for lien_bar in liens_bars:
    # Construire l'URL complète du bar
    url_bar = "https://www.schlouk-map.com" + lien_bar
    # Extraire les informations du bar
    informations_bar = extraire_informations_bar(url_bar)
    # Ajouter les informations extraites à la liste
    informations_bars.append(informations_bar)

for i, informations_bar in enumerate(informations_bars, 1):
    print(f"Informations du bar {i}:")
    for cle, valeur in informations_bar.items():
        if valeur is not None:
            if isinstance(valeur, list):  # Vérifier si la valeur est une liste
                valeur_sans_saut_ligne = ', '.join(valeur)  # Convertir la liste en chaîne de caractères
            else:
                valeur_sans_saut_ligne = valeur.replace('\n', '') if isinstance(valeur, str) else valeur  # Remplacer les caractères de saut de ligne par des chaînes vides
            print(f"{cle}: {valeur_sans_saut_ligne}")
    print()


Informations du bar 1:
Nom: Matabiau Décapsule Club
Heures_Ouverture: OuvertureLundi15:30 - 20:00Mardi13:00 - 23:00Mercredi13:00 - 23:00Jeudi13:00 - 23:00Vendredi13:00 - 23:00Samedi13:00 - 23:00DimancheFermé
Services: Restauration, Vente à emporter, Wifi gratuit, 🌬 Air climatisé, Fléchettes, Jeux de société
Tarifs: Happy HourStandardBière blonde(50cl)3,70 €4,90 €Bière blonde(25cl)- 2,80 €

Informations du bar 2:
Nom: Tom Pouce
Heures_Ouverture: OuvertureLundi10:30 - 20:00Mardi10:30 - 20:00Mercredi10:30 - 20:00Jeudi10:30 - 20:00Vendredi10:30 - 20:00Samedi10:30 - 20:00DimancheFermé
Services: Terrasse, Restauration, Vente à emporter, 🌬 Air climatisé, Accès & Toilettes Handicapés, DJ Mix, Chiens acceptés
Tarifs: Happy HourStandard1664 Blanche(50cl)5,50 €7,00 €

Informations du bar 3:
Nom: The Black Lion
Heures_Ouverture: OuvertureLundi08:30 - 02:00Mardi08:30 - 02:00Mercredi08:30 - 02:00Jeudi08:30 - 02:00Vendredi08:30 - 02:00Samedi08:30 - 03:00Dimanche12:00 - 02:00
Services: Terrasse, Resta

In [3]:
import pandas as pd

# Créer un DataFrame à partir des informations des bars
df = pd.DataFrame(informations_bars)

# Afficher le DataFrame
df

,Nom,Heures_Ouverture,Services,Tarifs
0,Matabiau Décapsule Club,Ouverture\nLundi\n\n15:30 - 20:00\n\nMardi\n\n...,"[Restauration, Vente à emporter, Wifi gratuit,...","Happy HourStandardBière blonde(50cl)\n3,70 €\n..."
1,Tom Pouce,Ouverture\nLundi\n\n10:30 - 20:00\n\nMardi\n\n...,"[Terrasse, Restauration, Vente à emporter, 🌬 A...","Happy HourStandard1664 Blanche(50cl)\n5,50 €\n..."
2,The Black Lion,Ouverture\nLundi\n\n08:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\..."
3,Barricot,Ouverture\nLundi\n\n12:00 - 22:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardHeineken(50cl)\n- \n8,20 €\n..."
4,La Maison,Ouverture\nLundi\n\n17:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Concerts / Musique Live]","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\..."
5,El Circo,Ouverture\nLundi\n\nFermé\n\nMardi\n\nFermé\n\...,"[Restauration, Concerts / Musique Live, DJ Mix]","61 Rue Pargaminières, 31000 Toulouse\n +33 5 3..."
6,Ô Boudu Pont,Ouverture\nLundi\n\n15:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration]","1 Rue de la République, 31300 Toulouse\n +33 9..."
7,Le Tchin,Ouverture\nLundi\n\n06:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Vente à emporter, Wif...","22 Rue Saint-Bernard, 31000 Toulouse\nTranspor..."
8,The Botanist Pub,Ouverture\nLundi\n\n08:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardMonaco(55cl)\n4,60 €\n\n5,70..."
9,Le Péry,Ouverture\nLundi\n\n10:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandard\nCéréalier\n(50cl)\n- \n6,5..."


In [4]:
df["Liens"] = liens_bars
df

,Nom,Heures_Ouverture,Services,Tarifs,Liens
0,Matabiau Décapsule Club,Ouverture\nLundi\n\n15:30 - 20:00\n\nMardi\n\n...,"[Restauration, Vente à emporter, Wifi gratuit,...","Happy HourStandardBière blonde(50cl)\n3,70 €\n...",/fr/places/matabiau-decapsule-club
1,Tom Pouce,Ouverture\nLundi\n\n10:30 - 20:00\n\nMardi\n\n...,"[Terrasse, Restauration, Vente à emporter, 🌬 A...","Happy HourStandard1664 Blanche(50cl)\n5,50 €\n...",/fr/places/tom-pouce
2,The Black Lion,Ouverture\nLundi\n\n08:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\...",/fr/places/the-black-lion-1
3,Barricot,Ouverture\nLundi\n\n12:00 - 22:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardHeineken(50cl)\n- \n8,20 €\n...",/fr/places/barricot
4,La Maison,Ouverture\nLundi\n\n17:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Concerts / Musique Live]","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\...",/fr/places/la-maison-1
5,El Circo,Ouverture\nLundi\n\nFermé\n\nMardi\n\nFermé\n\...,"[Restauration, Concerts / Musique Live, DJ Mix]","61 Rue Pargaminières, 31000 Toulouse\n +33 5 3...",/fr/places/el-circo
6,Ô Boudu Pont,Ouverture\nLundi\n\n15:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration]","1 Rue de la République, 31300 Toulouse\n +33 9...",/fr/places/o-boudu-pont
7,Le Tchin,Ouverture\nLundi\n\n06:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Vente à emporter, Wif...","22 Rue Saint-Bernard, 31000 Toulouse\nTranspor...",/fr/places/le-tchin
8,The Botanist Pub,Ouverture\nLundi\n\n08:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardMonaco(55cl)\n4,60 €\n\n5,70...",/fr/places/the-botanist-pub
9,Le Péry,Ouverture\nLundi\n\n10:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandard\nCéréalier\n(50cl)\n- \n6,5...",/fr/places/le-pery


### PAGE 2

In [5]:
import requests
from bs4 import BeautifulSoup

# URL de la page contenant les liens des bars
url_page2 = "https://www.schlouk-map.com/fr/cities/toulouse/happy-hour?page=2"

# Récupérer la page contenant les liens des bars
response_page2 = requests.get(url_page2)
soup_page2 = BeautifulSoup(response_page2.content, 'html.parser')

# Récupérer tous les liens des bars de la page
liens_bars2 = [a['href'] for a in soup_page2.find_all('a', class_='name')]

# Afficher les liens des bars
for lien2 in liens_bars2:
    print(lien2)

/fr/places/the-tower-of-london
/fr/places/la-cale-seche
/fr/places/g-bar
/fr/places/au-poele-de-la-bete
/fr/places/levasion
/fr/places/le-comptoir-2
/fr/places/le-bibent
/fr/places/el-chivito
/fr/places/mucho
/fr/places/dubliners
/fr/places/moloko
/fr/places/taparcero
/fr/places/chez-tonton
/fr/places/wanted-jack-saloon
/fr/places/le-petit-voisin
/fr/places/la-goulue
/fr/places/smoking-barrel
/fr/places/le-bierographe
/fr/places/l-episode
/fr/places/the-four-monkeys
/fr/places/le-biergarten-toulouse
/fr/places/lancienne-belgique
/fr/places/bar-des-zes
/fr/places/le-carbet-doc
/fr/places/pub-oclock
/fr/places/the-george-and-dragon
/fr/places/the-melting-pot-pub
/fr/places/les-tricheurs-1
/fr/places/baraka-jeux-1
/fr/places/dada
/fr/places/le-filochard


In [6]:
import requests
from bs4 import BeautifulSoup
import re

def extraire_informations_bar2(href_bar2):
    
    # Récupération de la page web du bar
    response2 = requests.get(href_bar2)
    soup2 = BeautifulSoup(response2.content, 'html.parser')

    # Étape 1 : Récupérer le nom du bar
    nom_bar2 = soup2.find('h1', class_='mb-0 d-inline-block').text.strip()

    # Étape 2 : Récupérer les heures d'ouverture
    heures_ouverture2 = soup2.find('div', class_="col-lg-6 mb-3 mb-md-0").text.strip()

    # Étape 4 : Récupérer les services (uniquement oui)
    services2 = []
    table_services2 = soup2.find('table', class_='table-services')
    if table_services2:
        for row in table_services2.find_all('tr'):
            service2 = row.find('td', class_='text-truncate').text.strip()
            disponibilite2 = row.find('td', class_='float-right').text.strip()
            if disponibilite2 == 'Oui':
                services2.append(service2)

    # Étape 5 : Récupérer les tarifs des boissons en happy hour/standard
    tarifs2 = soup2.find('div', class_= "position-relative").text.strip()

    # Retourner les informations
    return {
        'Nom': nom_bar2,
        'Heures_Ouverture': heures_ouverture2,
        'Services': services2,
        'Tarifs': tarifs2
    }

# Liste pour stocker les informations extraites de chaque bar
informations_bars2 = []

# Itérer sur chaque lien de bar
for lien_bar2 in liens_bars2:
    # Construire l'URL complète du bar
    url_bar2 = "https://www.schlouk-map.com" + lien_bar2
    # Extraire les informations du bar
    informations_bar2 = extraire_informations_bar2(url_bar2)
    # Ajouter les informations extraites à la liste
    informations_bars2.append(informations_bar2)

for i, informations_bar2 in enumerate(informations_bars2, 36):
    print(f"Informations du bar {i}:")
    for cle2, valeur2 in informations_bar2.items():
        if valeur is not None:
            if isinstance(valeur2, list):  # Vérifier si la valeur est une liste
                valeur_sans_saut_ligne2 = ', '.join(valeur2)  # Convertir la liste en chaîne de caractères
            else:
                valeur_sans_saut_ligne2 = valeur2.replace('\n', '') if isinstance(valeur2, str) else valeur2  # Remplacer les caractères de saut de ligne par des chaînes vides
            print(f"{cle2}: {valeur_sans_saut_ligne2}")
    print()

Informations du bar 36:
Nom: The Tower of London
Heures_Ouverture: OuvertureLundi17:00 - 00:00Mardi17:00 - 00:00Mercredi17:00 - 02:00Jeudi17:00 - 02:00Vendredi17:00 - 02:00Samedi15:00 - 03:00Dimanche15:00 - 00:00
Services: Terrasse, Diffusion de matchs, Fléchettes
Tarifs: Happy HourStandardCharlie Wells(50cl)5,20 €- Bombardier(50cl)5,50 €- aaaaaaaaaaaaa 4,00 €3,00 €aaaaaaaaaaaaaaaaaaa 1,00 €4,00 €aaaaaaaaaaaaaaaaaaa 7,00 €3,00 €aaaaaaaaaaaaaa 1,00 €1,00 €aaaaaaaaaaaaaa 4,00 €2,00 €aaaaaaaaaaaaaaa 8,00 €5,00 €aaaaaaaaaaaaaaa 1,00 €7,00 €aaaaaaaaaaaaaaaa 6,00 €3,00 €aaaaaaaaaaaaaa 5,00 €1,00 €aaaaaaaaaaaaaaaaa 4,00 €4,00 €aaaaaaaaaaaa 8,00 €4,00 €Pour voir les horaires et les prix de toutes les boissons en happy hour, télécharge l'appli Schlouk Map Scanner le QR CodeAppli gratuite  ·  4,8/5

Informations du bar 37:
Nom: La Cale Sèche
Heures_Ouverture: OuvertureLundi18:00 - 02:00Mardi18:00 - 02:00Mercredi18:00 - 02:00Jeudi18:00 - 02:00Vendredi18:00 - 02:00Samedi18:00 - 03:00DimancheFermé


In [7]:
import pandas as pd

# Créer un DataFrame à partir des informations des bars
df2 = pd.DataFrame(informations_bars2)

# Afficher le DataFrame
df2

,Nom,Heures_Ouverture,Services,Tarifs
0,The Tower of London,Ouverture\nLundi\n\n17:00 - 00:00\n\nMardi\n\n...,"[Terrasse, Diffusion de matchs, Fléchettes]","Happy HourStandardCharlie Wells(50cl)\n5,20 €\..."
1,La Cale Sèche,Ouverture\nLundi\n\n18:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Concerts / Musique Live]",Happy HourStandardBouteille de planteur(2cl)\n...
2,G-Bar,Ouverture\nLundi\n\nFermé\n\nMardi\n\n20:00 - ...,"[Terrasse, Restauration, Concerts / Musique Live]","Happy HourStandardBière(50cl)\n5,00 €\n\n- Coc..."
3,Au Poêle de la Bête,Ouverture\nLundi\n\n08:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardVerre de vin(17cl)\n2,00 €\n..."
4,L'Evasion,Ouverture\nLundi\n\n10:30 - 02:00\n\nMardi\n\n...,"[Restauration, Diffusion de matchs]","Happy HourStandardStella Artois(50cl)\n3,50 €\..."
5,Le Comptoir,Ouverture\nLundi\n\nFermé\n\nMardi\n\n17:30 - ...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardLicorne(50cl)\n- \n7,00 €"
6,Le Bibent,Ouverture\nLundi\n\n09:00 - 22:30\n\nMardi\n\n...,"[Terrasse, Restauration]","5 Place du Capitole, 31000 Toulouse"
7,El Chivito,Ouverture\nLundi\n\nFermé\n\nMardi\n\n18:00 - ...,"[Terrasse, Restauration]","15 Rue des Blanchers, 31000 Toulouse\n +33 5 6..."
8,Mucho,Ouverture\nLundi\n\nFermé\n\nMardi\n\n09:00 - ...,"[Terrasse, Restauration, Wifi gratuit, Diffusi...","9 Place du Pont Neuf, 31000 Toulouse\n +33 5 6..."
9,Dubliners,Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","46 Avenue Marcel Langer, 31400 Toulouse\n +33 ..."


In [8]:
df2["Liens"] = liens_bars2
df2

,Nom,Heures_Ouverture,Services,Tarifs,Liens
0,The Tower of London,Ouverture\nLundi\n\n17:00 - 00:00\n\nMardi\n\n...,"[Terrasse, Diffusion de matchs, Fléchettes]","Happy HourStandardCharlie Wells(50cl)\n5,20 €\...",/fr/places/the-tower-of-london
1,La Cale Sèche,Ouverture\nLundi\n\n18:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Concerts / Musique Live]",Happy HourStandardBouteille de planteur(2cl)\n...,/fr/places/la-cale-seche
2,G-Bar,Ouverture\nLundi\n\nFermé\n\nMardi\n\n20:00 - ...,"[Terrasse, Restauration, Concerts / Musique Live]","Happy HourStandardBière(50cl)\n5,00 €\n\n- Coc...",/fr/places/g-bar
3,Au Poêle de la Bête,Ouverture\nLundi\n\n08:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardVerre de vin(17cl)\n2,00 €\n...",/fr/places/au-poele-de-la-bete
4,L'Evasion,Ouverture\nLundi\n\n10:30 - 02:00\n\nMardi\n\n...,"[Restauration, Diffusion de matchs]","Happy HourStandardStella Artois(50cl)\n3,50 €\...",/fr/places/levasion
5,Le Comptoir,Ouverture\nLundi\n\nFermé\n\nMardi\n\n17:30 - ...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardLicorne(50cl)\n- \n7,00 €",/fr/places/le-comptoir-2
6,Le Bibent,Ouverture\nLundi\n\n09:00 - 22:30\n\nMardi\n\n...,"[Terrasse, Restauration]","5 Place du Capitole, 31000 Toulouse",/fr/places/le-bibent
7,El Chivito,Ouverture\nLundi\n\nFermé\n\nMardi\n\n18:00 - ...,"[Terrasse, Restauration]","15 Rue des Blanchers, 31000 Toulouse\n +33 5 6...",/fr/places/el-chivito
8,Mucho,Ouverture\nLundi\n\nFermé\n\nMardi\n\n09:00 - ...,"[Terrasse, Restauration, Wifi gratuit, Diffusi...","9 Place du Pont Neuf, 31000 Toulouse\n +33 5 6...",/fr/places/mucho
9,Dubliners,Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","46 Avenue Marcel Langer, 31400 Toulouse\n +33 ...",/fr/places/dubliners


In [9]:
df_concat = pd.concat([df, df2], ignore_index=True)
df_concat

,Nom,Heures_Ouverture,Services,Tarifs,Liens
0,Matabiau Décapsule Club,Ouverture\nLundi\n\n15:30 - 20:00\n\nMardi\n\n...,"[Restauration, Vente à emporter, Wifi gratuit,...","Happy HourStandardBière blonde(50cl)\n3,70 €\n...",/fr/places/matabiau-decapsule-club
1,Tom Pouce,Ouverture\nLundi\n\n10:30 - 20:00\n\nMardi\n\n...,"[Terrasse, Restauration, Vente à emporter, 🌬 A...","Happy HourStandard1664 Blanche(50cl)\n5,50 €\n...",/fr/places/tom-pouce
2,The Black Lion,Ouverture\nLundi\n\n08:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Diffusion de matchs, ...","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\...",/fr/places/the-black-lion-1
3,Barricot,Ouverture\nLundi\n\n12:00 - 22:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardHeineken(50cl)\n- \n8,20 €\n...",/fr/places/barricot
4,La Maison,Ouverture\nLundi\n\n17:30 - 02:00\n\nMardi\n\n...,"[Terrasse, Restauration, Concerts / Musique Live]","Happy HourStandardKronenbourg(50cl)\n4,00 €\n\...",/fr/places/la-maison-1
...,...,...,...,...,...
62,The Melting Pot Pub,Ouverture\nLundi\n\n12:00 - 02:00\n\nMardi\n\n...,"[Terrasse, Wifi gratuit, Diffusion de matchs, ...","Happy HourStandardStella Artois(50cl)\n4,70 €\...",/fr/places/the-melting-pot-pub
63,Les Tricheurs,Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...,[Jeux de société],"Happy HourStandardLicorne(50cl)\n5,00 €\n\n- \...",/fr/places/les-tricheurs-1
64,Baraka Jeux,Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...,"[Terrasse, Restauration]","Happy HourStandardStella Artois(50cl)\n3,90 €\...",/fr/places/baraka-jeux-1
65,Dada,Ouverture\nLundi\n\n18:00 - 00:00\n\nMardi\n\n...,"[Terrasse, Restauration, Wifi gratuit, Accès &...","Happy HourStandardMutzig(50cl)\n4,00 €\n\n- Pi...",/fr/places/dada


In [10]:
# Exporter le DataFrame au format CSV avec une tabulation comme séparateur
df_concat.to_csv('happyhours_toulouse.csv', sep='\t', index=False)

## PARTIE 2 : Analyse des Données


In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response_page.content, 'html.parser')
url = "https://www.schlouk-map.com/fr/cities/toulouse/happy-hour"
response = requests.get(url)
import re


price = soup.find_all('p', class_="text-truncate mb-2 text-secondary")
all_prices=[]
for item in price:
  motif=re.search(r'\d\,\d{2}',item.text)
  if motif:
    prix_motif=float(motif.group(0).replace(',','.'))
    all_prices.append(prix_motif)
  else:
    all_prices.append("NA")
print(all_prices)


balises = soup.find_all('a', class_="name")

# Liste pour stocker les noms des bars
noms_bars = []

# Boucle sur chaque balise pour extraire le texte
for balise in balises:
    nom_bar = balise.text.strip()
    noms_bars.append(nom_bar)

# Afficher les noms des bars
print(noms_bars)



dfPrix = pd.DataFrame({ 'Prix_Happy_Hours': all_prices, 'Noms' : noms_bars})
dfPrix

[3.7, 5.5, 4.0, 8.0, 4.0, 'NA', 'NA', 'NA', 4.6, 6.5, 9.0, 5.0, 5.0, 3.0, 4.5, 5.0, 4.5, 'NA', 3.0, 3.0, 3.0, 5.5, 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 5.8, 6.0, 4.2]
['Matabiau Décapsule Club', 'Tom Pouce', 'The Black Lion', 'Barricot', 'La Maison', 'El Circo', 'Ô Boudu Pont', 'Le Tchin', 'The Botanist Pub', 'Le Péry', 'Le Biergarten', 'Café Oz', 'The Frog & Rosbif', 'The Petit London', 'The Danu', 'Snapper Rock', 'The Hopscotch', 'Le Bistrologue', '1862 Artybar', 'Kraken Paradise', 'MAD (Meet and Drink)', 'Le Delicatessen', 'Bar Martini Club', 'Borriquito Loco', "L'Autruche", 'Sauvage Social Pub', 'Beer Mosaic', "L'Almanach Bistrot", 'Le Saint des Seins', 'La Tireuse', 'The London Town', 'La Tantina De Burgos', 'Rooster & Beer', 'Cacahuète', 'Au Fût et à Mesure', 'Café Populaire']


,Prix_Happy_Hours,Noms
0,3.7,Matabiau Décapsule Club
1,5.5,Tom Pouce
2,4.0,The Black Lion
3,8.0,Barricot
4,4.0,La Maison
5,NA,El Circo
6,NA,Ô Boudu Pont
7,NA,Le Tchin
8,4.6,The Botanist Pub
9,6.5,Le Péry


In [12]:
from bs4 import BeautifulSoup
soup2 = BeautifulSoup(response_page2.content, 'html.parser')
url2 = "https://www.schlouk-map.com/fr/cities/toulouse/happy-hour?page=2"
response2 = requests.get(url2)
import re


price2 = soup2.find_all('p', class_="text-truncate mb-2 text-secondary")
all_prices2=[]
for item2 in price2:
  motif2=re.search(r'\d\,\d{2}',item2.text)
  if motif2:
    prix_motif2=float(motif2.group(0).replace(',','.'))
    all_prices2.append(prix_motif2)
  else:
    all_prices2.append("NA")
print(all_prices2)

balises2 = soup2.find_all('a', class_="name")

# Liste pour stocker les noms des bars
noms_bars2 = []

# Boucle sur chaque balise pour extraire le texte
for balise2 in balises2:
    nom_bar2 = balise2.text.strip()
    noms_bars2.append(nom_bar2)

# Afficher les noms des bars
print(noms_bars2)



dfPrix2 = pd.DataFrame({ 'Prix_Happy_Hours': all_prices2, 'Noms' : noms_bars2})
dfPrix2

[5.2, 5.0, 5.0, 'NA', 3.5, 7.0, 'NA', 'NA', 'NA', 'NA', 'NA', 3.0, 2.6, 4.0, 3.0, 4.0, 'NA', 'NA', 'NA', 5.4, 6.0, 4.0, 7.0, 4.0, 4.0, 5.2, 4.7, 4.0, 3.9, 4.0, 3.0]
['The Tower of London', 'La Cale Sèche', 'G-Bar', 'Au Poêle de la Bête', "L'Evasion", 'Le Comptoir', 'Le Bibent', 'El Chivito', 'Mucho', 'Dubliners', 'Moloko', 'Taparcero', 'Chez Tonton', 'Wanted Jack Saloon', 'Le Petit Voisin', 'La Goulue', 'Smoking Barrel', 'Le Bièrographe', 'L Episode', 'The Four Monkeys', 'Le Biergarten Toulouse', "L'Ancienne Belgique", 'Bar Des Zés', "Le Carbet d'Oc", "Pub O'Clock", 'The George and Dragon', 'The Melting Pot Pub', 'Les Tricheurs', 'Baraka Jeux', 'Dada', 'Le Filochard']


,Prix_Happy_Hours,Noms
0,5.2,The Tower of London
1,5.0,La Cale Sèche
2,5.0,G-Bar
3,NA,Au Poêle de la Bête
4,3.5,L'Evasion
5,7.0,Le Comptoir
6,NA,Le Bibent
7,NA,El Chivito
8,NA,Mucho
9,NA,Dubliners


In [13]:
dfPrix['Prix_Happy_Hours'] = dfPrix['Prix_Happy_Hours'].replace('NA', 0).astype(float)
dfPrix

/var/folders/74/6gk6xbf50ybdlcwf5bzxpvp40000gn/T/ipykernel_61453/2279230812.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfPrix['Prix_Happy_Hours'] = dfPrix['Prix_Happy_Hours'].replace('NA', 0).astype(float)


,Prix_Happy_Hours,Noms
0,3.7,Matabiau Décapsule Club
1,5.5,Tom Pouce
2,4.0,The Black Lion
3,8.0,Barricot
4,4.0,La Maison
5,0.0,El Circo
6,0.0,Ô Boudu Pont
7,0.0,Le Tchin
8,4.6,The Botanist Pub
9,6.5,Le Péry


In [14]:
dfPrix2['Prix_Happy_Hours'] = dfPrix2['Prix_Happy_Hours'].replace('NA', 0).astype(float)
dfPrix2

/var/folders/74/6gk6xbf50ybdlcwf5bzxpvp40000gn/T/ipykernel_61453/3292599660.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfPrix2['Prix_Happy_Hours'] = dfPrix2['Prix_Happy_Hours'].replace('NA', 0).astype(float)


,Prix_Happy_Hours,Noms
0,5.2,The Tower of London
1,5.0,La Cale Sèche
2,5.0,G-Bar
3,0.0,Au Poêle de la Bête
4,3.5,L'Evasion
5,7.0,Le Comptoir
6,0.0,Le Bibent
7,0.0,El Chivito
8,0.0,Mucho
9,0.0,Dubliners


In [15]:
df_concatprix = pd.concat([dfPrix, dfPrix2], ignore_index=True)
df_concatprix

,Prix_Happy_Hours,Noms
0,3.7,Matabiau Décapsule Club
1,5.5,Tom Pouce
2,4.0,The Black Lion
3,8.0,Barricot
4,4.0,La Maison
...,...,...
62,4.7,The Melting Pot Pub
63,4.0,Les Tricheurs
64,3.9,Baraka Jeux
65,4.0,Dada


In [16]:
# Supprimer les lignes avec des prix nuls
df_final_non_null = df_concatprix[df_concatprix['Prix_Happy_Hours'] != 0]


In [17]:
df_final_non_null

,Prix_Happy_Hours,Noms
0,3.7,Matabiau Décapsule Club
1,5.5,Tom Pouce
2,4.0,The Black Lion
3,8.0,Barricot
4,4.0,La Maison
8,4.6,The Botanist Pub
9,6.5,Le Péry
10,9.0,Le Biergarten
11,5.0,Café Oz
12,5.0,The Frog & Rosbif


In [18]:
# Calculer le prix moyen des boissons pendant les heures de happy hour en excluant les valeurs nulles
prix_moyen_non_null = df_final_non_null['Prix_Happy_Hours'].mean().round(2)

print("Le prix moyen des boissons pendant les heures de happy hour est de :", prix_moyen_non_null, "€")


Le prix moyen des boissons pendant les heures de happy hour est de : 4.66 €


In [19]:
# Trouver le prix minimum et le nom du bar associé
prix_min_filtre = df_final_non_null['Prix_Happy_Hours'].min()
nom_bar_prix_min_filtre = df_final_non_null.loc[df_final_non_null['Prix_Happy_Hours'] == prix_min_filtre, 'Noms'].iloc[0]
prix_max_filtre = df_final_non_null['Prix_Happy_Hours'].max()
nom_bar_prix_max_filtre = df_final_non_null.loc[df_final_non_null['Prix_Happy_Hours'] == prix_max_filtre, 'Noms'].iloc[0]

# Afficher le résultat avec le nom du bar
print("Le prix maximum de", prix_max_filtre, "€ est associé au bar :", nom_bar_prix_max_filtre)

# Afficher le résultat avec le nom du bar
print("Le prix minimum de", prix_min_filtre, "€ est associé au bar :", nom_bar_prix_min_filtre)

Le prix maximum de 9.0 € est associé au bar : Le Biergarten
Le prix minimum de 2.6 € est associé au bar : Chez Tonton


In [20]:
df_trie = df_final_non_null.sort_values(by='Prix_Happy_Hours')
df_trie

,Prix_Happy_Hours,Noms
48,2.6,Chez Tonton
19,3.0,Kraken Paradise
13,3.0,The Petit London
18,3.0,1862 Artybar
50,3.0,Le Petit Voisin
47,3.0,Taparcero
20,3.0,MAD (Meet and Drink)
66,3.0,Le Filochard
40,3.5,L'Evasion
0,3.7,Matabiau Décapsule Club


In [21]:
# Sélectionner les dix premières entrées (les moins chères)
cinq_bars_moins_chers = df_trie.head(10)

# Extraire les noms des bars
noms_cinq_bars_moins_chers = cinq_bars_moins_chers['Noms'].tolist()

# Afficher les noms des cinq bars les moins chers
print("Les bars avec les prix les moins chers à Toulouse sont :")
for index, row in cinq_bars_moins_chers.iterrows():
    print("- Bar :", row['Noms'])
    print("  Prix Happy Hours :", row['Prix_Happy_Hours'], "€")

Les bars avec les prix les moins chers à Toulouse sont :
- Bar : Chez Tonton
  Prix Happy Hours : 2.6 €
- Bar : Kraken Paradise
  Prix Happy Hours : 3.0 €
- Bar : The Petit London
  Prix Happy Hours : 3.0 €
- Bar : 1862 Artybar
  Prix Happy Hours : 3.0 €
- Bar : Le Petit Voisin
  Prix Happy Hours : 3.0 €
- Bar : Taparcero
  Prix Happy Hours : 3.0 €
- Bar : MAD (Meet and Drink)
  Prix Happy Hours : 3.0 €
- Bar : Le Filochard
  Prix Happy Hours : 3.0 €
- Bar : L'Evasion
  Prix Happy Hours : 3.5 €
- Bar : Matabiau Décapsule Club
  Prix Happy Hours : 3.7 €


## PARTIE 3 : Rapport par E-mail 


In [22]:
import smtplib
import email.mime

In [35]:
from jinja2 import Environment, FileSystemLoader

# Charger le répertoire contenant les templates
env = Environment(loader=FileSystemLoader('/Users/celiamarty/Desktop/HappyHourProject/templates'))

env


In [36]:
# Charger le template HTML
template = env.get_template('report_templatee.html')
template

<Template 'report_templatee.html'>

In [37]:
# Simuler des données (remplacer par les données réelles extraites et analysées)
bars = [
    {'name': 'Chez Tonton', 'address': '16 Place Saint-Pierre, 31000 Toulouse', 'happy_hour_price': 2.60, 'normal_price': 3.50, 'savings': 1.10},
    {'name': '1862 Artybar', 'address': '24 Rue Nicolas Bachelier, 31000 Toulouse', 'happy_hour_price': 3.0, 'normal_price': 6.0, 'savings': 3.0},
    {'name': 'Le Petit Voisin', 'address': '37 Rue Peyrolières, 31000 Toulouse', 'happy_hour_price': 3.0, 'normal_price': 4.0, 'savings': 1.0},
    {'name': 'MAD (Meet And Drink)', 'address': '4 Allées Charles de Fitte, 31300 Toulouse', 'happy_hour_price': 3.0, 'normal_price': 5.0, 'savings': 2.0},
    {'name': 'Matabiau Décapsule Club', 'address': '74 Rue Matabiau, 31000 Toulouse', 'happy_hour_price': 3.70, 'normal_price': 4.90, 'savings': 1.20}
]

# Rendre le template avec les données
html_output = template.render(bars=bars)

# Écrire le résultat dans un fichier HTML
with open('/Users/celiamarty/Desktop/WEBSCRAPPING/templates/report_templatee.html', 'w') as f:
    f.write(html_output)

In [38]:
smtpObj = smtplib.SMTP


In [39]:
help(smtplib)

Help on module smtplib:

NAME
    smtplib - SMTP/ESMTP client class.

MODULE REFERENCE
    https://docs.python.org/3.9/library/smtplib
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This should follow RFC 821 (SMTP), RFC 1869 (ESMTP), RFC 2554 (SMTP
    Authentication) and RFC 2487 (Secure SMTP over TLS).
    
    Notes:
    
    Please remember, when doing ESMTP, that the names of the SMTP service
    extensions are NOT the same thing as the option keywords for the RCPT
    and MAIL commands!
    
    Example:
    
      >>> import smtplib
      >>> s=smtplib.SMTP("localhost")
      >>> print(s.help())
      This is Sendmail version 8.8.4
      Topics:
          HELO    EHLO    MAIL    RCPT    D

In [40]:
from mailtrap import Mail, Address, MailtrapClient

In [41]:
import mailtrap as mt

mail = mt.Mail(
    sender=mt.Address(email="mailtrap@demomailtrap.com", name="Mailtrap Test"),
    to=[mt.Address(email="celiamartypro@gmail.com")],
    subject="TOP 5 BARS!",
    text="TOP 5 des meilleurs deals en Happy Hours à Toulouse!",
    category="Integration Test",
    html="""
    <!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <title>Rapport des Happy Hours à Toulouse</title>
    <style>
        body { font-family: Arial, sans-serif; }
        table { width: 100%; border-collapse: collapse; }
        th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
        th { background-color: #f2f2f2; }
    </style>
</head>
<body>
    <h1>Rapport des Happy Hours à Toulouse</h1>
    <h2>Top 5 des Meilleurs Deals</h2>
    <table>
        <tr>
            <th>Nom du Bar</th>
            <th>Adresse</th>
            <th>Prix Happy Hour</th>
            <th>Prix Normal</th>
            <th>Économie</th>
        </tr>
<tr>
            <td>Chez Tonton</td>
            <td>16 Place Saint-Pierre, 31000 Toulouse</td>
            <td>2.6 €</td>
            <td>3.5 €</td>
            <td>1.1 €</td>
        </tr>

        <tr>
            <td>1862 Artybar</td>
            <td>24 Rue Nicolas Bachelier, 31000 Toulouse</td>
            <td>3.0 €</td>
            <td>6.0 €</td>
            <td>3.0 €</td>
        </tr>

        <tr>
            <td>Le Petit Voisin</td>
            <td>37 Rue Peyrolières, 31000 Toulouse</td>
            <td>3.0 €</td>
            <td>4.0 €</td>
            <td>1.0 €</td>
        </tr>

        <tr>
            <td>MAD (Meet And Drink)</td>
            <td>4 Allées Charles de Fitte, 31300 Toulouse</td>
            <td>3.0 €</td>
            <td>5.0 €</td>
            <td>2.0 €</td>
        </tr>

        <tr>
<td>Matabiau Décapsule Club</td>
            <td>74 Rue Matabiau, 31000 Toulouse</td>
            <td>3.7 €</td>
            <td>4.9 €</td>
            <td>1.2 €</td>
        </tr>

    </table>
</body>
</html>
    """
)

client = mt.MailtrapClient(token="2eb63b46a5db29cfd1df99b657d472a0")
client.send(mail)


{'success': True, 'message_ids': ['b3aa66d0-13a7-11ef-0040-f189db6f9d44']}

## Partie 4 : Cartographie (Bonus)

In [42]:
import pandas as pd

data_carte1= {
    "Nom du bar": [
        "Matabiau Décapsule Club", "Tom Pouce", "The Black Lion", "Barricot", "La Maison", 
        "El Circo", "Ô Boudu Pont", "Le Tchin", "The Botanist Pub", "Le Péry", 
        "Le Biergarten", "Café Oz", "The Frog & Rosbif", "The Petit London", "The Danu", 
        "Snapper Rock", "The Hopscotch", "Le Bistrologue", "1862 Artybar", "Kraken Paradise", 
        "MAD (Meet and Drink)", "Le Delicatessen", "Bar Martini Club", "Borriquito Loco", 
        "L'Autruche", "Sauvage Social Pub", "Beer Mosaic", "L'Almanach Bistrot", 
        "Le Saint des Seins", "La Tireuse", "The London Town", "La Tantina De Burgos", 
        "Rooster & Beer", "Cacahuète", "Au Fût et à Mesure", "Café Populaire", 
        "The Tower of London", "La Cale Sèche", "G-Bar", "Au Poêle de la Bête", 
        "L'Evasion", "Le Comptoir", "Le Bibent", "El Chivito", "Mucho", "Dubliners", 
        "Moloko", "Taparcero", "Chez Tonton", "Wanted Jack Saloon", "Le Petit Voisin", 
        "La Goulue", "Smoking Barrel", "Le Bièrographe", "L Episode", "The Four Monkeys", 
        "Le Biergarten Toulouse", "L'Ancienne Belgique", "Bar Des Zés", "Le Carbet d'Oc", 
        "Pub O'Clock", "The George and Dragon", "The Melting Pot Pub", "Les Tricheurs", 
        "Baraka Jeux", "Dada", "Le Filochard"
    ],
    "Adresse": [
        "74 Rue Matabiau, 31000 Toulouse", "4 Place Roger Salengro, 31000 Toulouse", 
        "4 Allées Paul Feuga, 31000 Toulouse", "8 Esplanade Compans Caffarelli, 31000 Toulouse", 
        "9 Rue Gabriel Pérri, 31000 Toulouse", "61 Rue Pargaminières, 31000 Toulouse", 
        "1 Rue de la République, 31300 Toulouse", "22 Rue Saint-Bernard, 31000 Toulouse", 
        "33 Boulevard Maréchal Leclerc, 31000 Toulouse", "22 Rue Gabriel Péri, 31000 Toulouse", 
        "60 Grand-Rue Saint-Michel, 31400 Toulouse", "1 Rue Gabriel Péri, 31000 Toulouse", 
        "14 Rue de l'Industrie, 31000 Toulouse", "7 Rue Riquet, 31000 Toulouse", 
        "9 Rue du Pont Guilhemery, 31000 Toulouse", "4 Rue Gabriel Péri, 31000 Toulouse", 
        "3 Rue Bour Lormian, 31000 Toulouse", "2 Place du Ravelin, 31300 Toulouse", 
        "24 Rue Nicolas Bachelier, 31000 Toulouse", "34 Rue de la Colombette, 31000 Toulouse", 
        "4 Allées Charles de Fitte, 31300 Toulouse", "11 bis Rue Riquet, 31000 Toulouse", 
        "9 Rue d'Austerlitz, 31000 Toulouse", "25 Rue des Paradoux, 31000 Toulouse", 
        "1 Rue André Mercadier, 31000 Toulouse", "11 Place Etienne Esquirol, 31000 Toulouse", 
        "7bis Rue Pierre-Paul Riquet, 31000 Toulouse", "53 Rue de la République, 31300 Toulouse", 
        "5 Place Saint-Pierre, 31000 Toulouse", "24 Rue Pargaminières, 31000 Toulouse", 
        "14 Rue des Prêtres, 31000 Toulouse", "27 Avenue de la Garonnette, 31000 Toulouse", 
        "100 Rue Riquet, 31000 Toulouse", "21 Place de l'Estrapade, 31300 Toulouse", 
        "20 Rue Gabriel Péri, 31000 Toulouse", "9 Rue de la Colombette, 31000 Toulouse", 
        "39 Grande Rue Saint-Michel, 31400 Toulouse", "41 Rue Léon Gambetta, 31000 Toulouse", 
        "10 Rue du Pont Montaudran, 31000 Toulouse", "56 Rue d'Aubuisson, 31000 Toulouse", 
        "29 Grande Rue Saint-Michel, 31400 Toulouse", "12 Avenue de Grande Bretagne, 31300 Toulouse", 
        "5 Place du Capitole, 31000 Toulouse", "15 Rue des Blanchers, 31000 Toulouse", 
        "9 Place du Pont Neuf, 31000 Toulouse", "46 Avenue Marcel Langer, 31400 Toulouse", 
        "6 Rue Joutx Aigues, 31000 Toulouse", "63 Rue Pargaminières, 31000 Toulouse", 
        "16 Place Saint-Pierre, 31000 Toulouse", "2 Rue Raymond IV, 31000 Toulouse", 
        "37 Rue Peyrolières, 31000 Toulouse", "43 Boulevard Lascrosses, 31000 Toulouse", 
        "5 Place Jean Diebold, 31300 Toulouse", "12 Rue des Paradoux, 31000 Toulouse", 
        "3 Rue des Blanchers, 31000 Toulouse", "7 Rue de Metz, 31000 Toulouse", 
        "60 Grande Rue Saint-Michel, 31400 Toulouse", "16 Rue de la Trinité, 31000 Toulouse", 
        "49 Rue Pargaminières, 31000 Toulouse", "4 Rue des Filatiers, 31000 Toulouse", 
        "21 Boulevard de Strasbourg, 31000 Toulouse", "1 Place du Peyrou, 31000 Toulouse", 
        "26 Boulevard de Strasbourg, 31000 Toulouse", "34 Rue des Blanchers, 31000 Toulouse", 
        "1 Boulevard de la Gare, 31500 Toulouse", "27 Avenue Honoré Serres, 31000 Toulouse", 
        "8 Place du Pont Neuf, 31000 Toulouse"
    ]
}

df_carte = pd.DataFrame(data_carte1)
df_carte

,Nom du bar,Adresse
0,Matabiau Décapsule Club,"74 Rue Matabiau, 31000 Toulouse"
1,Tom Pouce,"4 Place Roger Salengro, 31000 Toulouse"
2,The Black Lion,"4 Allées Paul Feuga, 31000 Toulouse"
3,Barricot,"8 Esplanade Compans Caffarelli, 31000 Toulouse"
4,La Maison,"9 Rue Gabriel Pérri, 31000 Toulouse"
...,...,...
62,The Melting Pot Pub,"26 Boulevard de Strasbourg, 31000 Toulouse"
63,Les Tricheurs,"34 Rue des Blanchers, 31000 Toulouse"
64,Baraka Jeux,"1 Boulevard de la Gare, 31500 Toulouse"
65,Dada,"27 Avenue Honoré Serres, 31000 Toulouse"


In [31]:
df_carte = pd.merge(df_carte, df_concat[['Heures_Ouverture']], left_index=True, right_index=True)
df_carte

,Nom du bar,Adresse,Heures_Ouverture
0,Matabiau Décapsule Club,"74 Rue Matabiau, 31000 Toulouse",Ouverture\nLundi\n\n15:30 - 20:00\n\nMardi\n\n...
1,Tom Pouce,"4 Place Roger Salengro, 31000 Toulouse",Ouverture\nLundi\n\n10:30 - 20:00\n\nMardi\n\n...
2,The Black Lion,"4 Allées Paul Feuga, 31000 Toulouse",Ouverture\nLundi\n\n08:30 - 02:00\n\nMardi\n\n...
3,Barricot,"8 Esplanade Compans Caffarelli, 31000 Toulouse",Ouverture\nLundi\n\n12:00 - 22:00\n\nMardi\n\n...
4,La Maison,"9 Rue Gabriel Pérri, 31000 Toulouse",Ouverture\nLundi\n\n17:30 - 02:00\n\nMardi\n\n...
...,...,...,...
62,The Melting Pot Pub,"26 Boulevard de Strasbourg, 31000 Toulouse",Ouverture\nLundi\n\n12:00 - 02:00\n\nMardi\n\n...
63,Les Tricheurs,"34 Rue des Blanchers, 31000 Toulouse",Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...
64,Baraka Jeux,"1 Boulevard de la Gare, 31500 Toulouse",Ouverture\nLundi\n\n17:00 - 01:00\n\nMardi\n\n...
65,Dada,"27 Avenue Honoré Serres, 31000 Toulouse",Ouverture\nLundi\n\n18:00 - 00:00\n\nMardi\n\n...


In [32]:
df_carte = pd.merge(df_carte, df_final_non_null[['Prix_Happy_Hours']], left_index=True, right_index=True)
df_carte

,Nom du bar,Adresse,Heures_Ouverture,Prix_Happy_Hours
0,Matabiau Décapsule Club,"74 Rue Matabiau, 31000 Toulouse",Ouverture\nLundi\n\n15:30 - 20:00\n\nMardi\n\n...,3.7
1,Tom Pouce,"4 Place Roger Salengro, 31000 Toulouse",Ouverture\nLundi\n\n10:30 - 20:00\n\nMardi\n\n...,5.5
2,The Black Lion,"4 Allées Paul Feuga, 31000 Toulouse",Ouverture\nLundi\n\n08:30 - 02:00\n\nMardi\n\n...,4.0
3,Barricot,"8 Esplanade Compans Caffarelli, 31000 Toulouse",Ouverture\nLundi\n\n12:00 - 22:00\n\nMardi\n\n...,8.0
4,La Maison,"9 Rue Gabriel Pérri, 31000 Toulouse",Ouverture\nLundi\n\n17:30 - 02:00\n\nMardi\n\n...,4.0
8,The Botanist Pub,"33 Boulevard Maréchal Leclerc, 31000 Toulouse",Ouverture\nLundi\n\n08:00 - 02:00\n\nMardi\n\n...,4.6
9,Le Péry,"22 Rue Gabriel Péri, 31000 Toulouse",Ouverture\nLundi\n\n10:00 - 02:00\n\nMardi\n\n...,6.5
10,Le Biergarten,"60 Grand-Rue Saint-Michel, 31400 Toulouse",Ouverture\nLundi\n\n17:00 - 02:00\n\nMardi\n\n...,9.0
11,Café Oz,"1 Rue Gabriel Péri, 31000 Toulouse",Ouverture\nLundi\n\n16:00 - 02:00\n\nMardi\n\n...,5.0
12,The Frog & Rosbif,"14 Rue de l'Industrie, 31000 Toulouse",Ouverture\nLundi\n\n17:00 - 00:00\n\nMardi\n\n...,5.0


In [33]:
from geopy.geocoders import Nominatim
import folium
from geopy.exc import GeocoderUnavailable

In [43]:
geolocator = Nominatim(user_agent="my_geocoder")

toulouse_location = geolocator.geocode("Toulouse, France")
print(toulouse_location)


m = folium.Map(location=[toulouse_location.latitude, toulouse_location.longitude], zoom_start=14)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Toulouse%2C+France&format=json&limit=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))

In [ ]:
for nom, adresse, Prix_Happy_Hours, Heures_Ouverture in zip(df_carte['Nom du bar'], df_carte['Adresse'], df_carte['Prix_Happy_Hours'], df_carte['Heures_Ouverture']):
    try:
        # Géocodage de l'adresse pour obtenir les coordonnées GPS
        location = geolocator.geocode(adresse)
        if location:
            # Création du texte pour le popup (nom du bar avec un lien et prix)
            popup_text = f'<strong>Nom:</strong>{nom}</a><br>{adresse}<br><strong>Prix happy hour:</strong>{Prix_Happy_Hours}€<br><strong>Horaires:</strong>{Heures_Ouverture}<br>'
            # Ajout d'un marqueur sur la carte avec les coordonnées GPS et le popup contenant le nom du bar, l'adresse, les prix, les horaires et les services
            folium.Marker([location.latitude, location.longitude], popup=popup_text,
                          tooltip=nom).add_to(m)
    except GeocoderUnavailable:
        print("Le service de géocodage est indisponible. Veuillez réessayer plus tard.")

m.save('map.html')

Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plus tard.
Le service de géocodage est indisponible. Veuillez réessayer plu

NameError: name 'm' is not defined

In [ ]:
m